In [1]:
import numpy as np
import igl

In [75]:
%%bash
m=~/Applications/meshgeometry/meshgeometry_mac
$m -iformat pial -i /Applications/_Neuro/freesurfer/subjects/fsaverage/surf/lh.inflated -o lh.inflated.ply
$m -iformat pial -i /Applications/_Neuro/freesurfer/subjects/fsaverage/surf/rh.inflated -o rh.inflated.ply

In [90]:
vw,fw=igl.read_triangle_mesh("./lrh.ply")
vl,fl=igl.read_triangle_mesh("./lh.inflated.ply")
vr,fr=igl.read_triangle_mesh("./rh.inflated.ply")

In [91]:
vl = vl - np.array([39,0,0])
vr = vr + np.array([39,0,0])

In [92]:
f=open("lh.corr.txt")
tmp=f.read().split("\n")
f.close()
ltmp = [x.split(" ") for x in tmp[:-1]]

f=open("rh.corr.txt")
tmp=f.read().split("\n")
f.close()
rtmp = [x.split(" ") for x in tmp[:-1]]

In [93]:
lut = -1*np.ones((len(ltmp)+len(rtmp),2), dtype=np.int)
for row in ltmp:
    if row[1] != 'NA':
        hindex = int(row[0])
        windex = int(row[1])
        lut[windex,0] = hindex
        lut[windex,1] = 1
for row in rtmp:
    if row[1] != 'NA':
        hindex = int(row[0])
        windex = int(row[1])
        lut[windex,0] = hindex
        lut[windex,1] = 2

In [94]:
fi = []
nonAssignedCount = 0
for f in fw:
    if lut[f[0],0] > -1 and lut[f[1],0] > -1 and lut[f[2],0] > -1:
        fi.append(f)
    else:
        nonAssignedCount+=1
fi=np.array(fi);
print(nonAssignedCount, "non assigned faces")

0 non assigned faces


In [95]:
vi = np.zeros(vw.shape)
for i in range(len(vw)):
    hem = lut[i,1]
    ind = lut[i,0]
    if hem == 1:
        vi[i,:] = vl[ind,:]
    elif hem ==2:
        vi[i,:] = vr[ind,:]

In [96]:
igl.write_triangle_mesh("lrh.inflated.ply",vi,fi)

True